In [1]:
import numpy as np
import os
import sys
#import rasterio as rst
from osgeo import gdal
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from skimage import data, transform
from skimage.util import img_as_ubyte
from skimage.morphology import disk
from skimage.filters import thresholding,rank
import rasterio as rio
from scipy import stats
import datetime
from skimage.morphology import rectangle,square
import pickle 


/opt/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Helpful functions used in classification, image IO, etc. 

In [3]:
# A list of "random" colors (for a nicer output)
COLORS = ["#000000", "#FFFF00"]# , "#1CE6FF", "#FF34FF", "#FF4A46", "#008941"]


def create_mask_from_vector(vector_data_path, cols, rows, geo_transform,
                            projection, target_value=1):
    """Rasterize the given vector (wrapper for gdal.RasterizeLayer)."""
    data_source = gdal.OpenEx(vector_data_path, gdal.OF_VECTOR)
    layer = data_source.GetLayer(0)
    driver = gdal.GetDriverByName('MEM')  # In memory dataset
    target_ds = driver.Create('', cols, rows, 1, gdal.GDT_UInt16)
    target_ds.SetGeoTransform(geo_transform)
    target_ds.SetProjection(projection)
    gdal.RasterizeLayer(target_ds, [1], layer, burn_values=[target_value])
    return target_ds


def vectors_to_raster(file_paths, rows, cols, geo_transform, projection):
    """Rasterize the vectors in the given directory in a single image."""
    labeled_pixels = np.zeros((rows, cols))
    for i, path in enumerate(file_paths):
        label = i+1
        ds = create_mask_from_vector(path, cols, rows, geo_transform, projection, target_value=label)
        band = ds.GetRasterBand(1)
        labeled_pixels += band.ReadAsArray()
        ds = None
    return labeled_pixels


def write_geotiff(fname, data, geo_transform, projection):
    """Create a GeoTIFF file with the given data."""
    driver = gdal.GetDriverByName('GTiff')
    rows, cols = data.shape
    dataset = driver.Create(fname, cols, rows, 1, gdal.GDT_Byte)
    dataset.SetGeoTransform(geo_transform)
    dataset.SetProjection(projection)
    band = dataset.GetRasterBand(1)
    band.WriteArray(data)
    dataset = None  # Close the file

#sys.path.append('/media/nero/DATADRIVE1')


This uses a training dataset, consiting of folders of one-per-image and two shapefiles (perm/imperm) to create a binary permeability classifier. This can be updated to use the city-index with "type" field on the zoneing information so we can better train classifiers for each urban zone

In [ ]:
classifier = RandomForestClassifier(n_jobs=-1)
# Allow division by zero
np.seterr(divide='ignore', invalid='ignore')

working_directory = '/media/DATADRIVE/permability/train'
for root, dirs, files in os.walk(working_directory, topdown=True):
    shapefiles = [os.path.join(root,f)  for f in files if f.endswith('.shp')]
    image =  [os.path.join(root,f)  for f in files if f.endswith('.tif')]
    if shapefiles and image:#e  and  root.startswith('M'):
        print(root,image)
        raster_dataset = gdal.Open(image[0], gdal.GA_ReadOnly)
        
        geo_transform = raster_dataset.GetGeoTransform()
        proj = raster_dataset.GetProjectionRef()
        bands_data = []
        
        for b in range(1, raster_dataset.RasterCount+1):
            band = raster_dataset.GetRasterBand(b)
            bands_data.append(band.ReadAsArray())
#         b4 = bands_data[3]
#         b3 = bands_data[0]
#        #bands_data = np.dstack(bands_data)

#         ndvi = ((b4.astype(np.float32) - b3.astype(np.float32)) / (b4 + b3))*100
#         bands_data.append(ndvi)
#         bands_data.pop(0)
#         bands_data.pop(1)
#        bands_data.pop(3)
     #   print(bands_data)
        bands_data = np.dstack(bands_data)
        bands_data = np.nan_to_num(bands_data)
        rows, cols, n_bands = bands_data.shape
        print("Training: "+root)
        labeled_pixels = vectors_to_raster(shapefiles, rows, cols, geo_transform,proj)
        is_train = np.nonzero(labeled_pixels)
        training_labels = labeled_pixels[is_train]
        training_samples = bands_data[is_train]
        classifier.fit(training_samples, training_labels)
#saves trained classifier to be reused later
f = open('/media/DATADRIVE/permability/ndf_perm_classifier_%.pkl'%,'wb')
pickle.dump(classifier,f)
f.close()

Classification on unseen images. Loops through a working_directory, creates ndvi band, and classifies permeability

In [8]:
#classifier = pickle.load(open('/media/DATADRIVE/imagery/suburban_a_ndfclassifier_022618.pkl','rb'))
#this loads pre-trained model. We'd Ideally have one for each type of urban zone - e.g., industrial etc
#classifier = pickle.load(open('/media/DATADRIVE/permability/ndf_perm_classifier_181118.pkl','rb'))
working_directory = '/media/DATADRIVE/imagery/raw_tiffs_2017/'
for root, dirs, files in os.walk(working_directory, topdown=True):
    for f in files:
        outfile = f.split('.')[0]+'_9classified.tif'
        if f.endswith('.tif') and not os.path.isfile(os.path.join('/media/DATADRIVE/permability/out_tifs/',outfile)):
            print(os.path.join(root,f))
            raster_dataset = gdal.Open(os.path.join(root,f), gdal.GA_ReadOnly)
            geo_transform = raster_dataset.GetGeoTransform()
            proj = raster_dataset.GetProjectionRef()
            bands_data = []
            for b in range(1, raster_dataset.RasterCount+1):
                band = raster_dataset.GetRasterBand(b)
                bands_data.append(band.ReadAsArray())
            b4 = bands_data[3]
            b3 = bands_data[0]
            ndvi = ((b4.astype(np.float32) - b3.astype(np.float32)) / (b4 + b3))*100
            bands_data.append(ndvi)
#             bands_data.pop(0)
#             bands_data.pop(1)
#             bands_data.pop(2)
            bands_data = np.dstack(bands_data)
            rows, cols, n_bands = bands_data.shape
            n_samples = rows*cols
            flat_pixels = bands_data.reshape((n_samples, n_bands))
            classifier.n_jobs=-1
            result = classifier.predict(np.nan_to_num(flat_pixels))
            classification = result.reshape((rows, cols))
#             classification3 =rank.modal(classification.astype(np.int8), square(3))
#             outfile = f.split('.')[0]+'_3classified.tif'
#             write_geotiff('./workspace/classified_tiffs/'+outfile, classification3, geo_transform, proj)
#             classification9 =rank.modal(classification.astype(np.int8), square(9))
#             outfile = image[0].split('.')[0]+'_9classified.tif'
#             write_geotiff(root+outfile, classification9, geo_transform, proj)
            classification12 =rank.modal(classification.astype(np.int8), square(9))
            #classification12[classification12 >2] = 1 #any value greater than 2 is assigned 1 -impermeable
            #classification12[classification12 <= 2] = 0 #any value with 2 are assigned 0 - permeable
            print(classification12)
            write_geotiff('/media/DATADRIVE/permability/out_tifs/'+outfile, classification12, geo_transform, proj)


This is a classify function to run if we want do multiprocessing. Note there was some errors with loading the individual modeals in each thread and benched this approach for now. 

In [3]:
def classify_images(fs):
#     classifier = pickle.load(open('/media/DATADRIVE/imagery/suburban_a_ndfclassifier_022618.pkl','rb'))
    for f in fs:
        root = '/media/DATADRIVE/imagery/raw_tiffs_2017/'
        print(os.path.join(root,f))
        raster_dataset = gdal.Open(os.path.join(root,f), gdal.GA_ReadOnly)
        print(f)
        geo_transform = raster_dataset.GetGeoTransform()
        proj = raster_dataset.GetProjectionRef()
        bands_data = []
        for b in range(1, raster_dataset.RasterCount+1):
            band = raster_dataset.GetRasterBand(b)
            bands_data.append(band.ReadAsArray())
#         b4 = bands_data[3]
#         b3 = bands_data[0]

            ndvi = ((b4.astype(np.float32) - b3.astype(np.float32)) / (b4 + b3))*100
            bands_data.append(ndvi)
#             bands_data.pop(0)
#             bands_data.pop(1)
#        bands_data.pop(3)
        bands_data = np.dstack(bands_data)
        rows, cols, n_bands = bands_data.shape
        n_samples = rows*cols
        flat_pixels = bands_data.reshape((n_samples, n_bands))
        classifier.n_jobs=2
        result = classifier.predict(np.nan_to_num(flat_pixels))
        classification = result.reshape((rows, cols))
#             classification3 =rank.modal(classification.astype(np.int8), square(3))
#             outfile = f.split('.')[0]+'_3classified.tif'
#             write_geotiff('./workspace/classified_tiffs/'+outfile, classification3, geo_transform, proj)
#             classification9 =rank.modal(classification.astype(np.int8), square(9))
#             outfile = image[0].split('.')[0]+'_9classified.tif'
#             write_geotiff(root+outfile, classification9, geo_transform, proj)
        classification12 =rank.modal(classification.astype(np.int8), square(9))
        outfile = f.split('.')[0]+'_9classified.tif'
        print('/media/DATADRIVE/permability/out_tifs/'+outfile)
        write_geotiff('/media/DATADRIVE/permability/out_tifs/'+outfile, classification12, geo_transform, proj)


In [ ]:
from multiprocessing import Pool

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]
fs = []
working_directory = '/media/DATADRIVE/imagery/raw_tiffs_2017/'
for root, dirs, files in os.walk(working_directory, topdown=True):
    for f in files:
        if f.endswith('.tif'):
            fs.append(f)
if __name__ == '__main__':
    pool = Pool(6)
    for res in pool.map(classify_images,chunks(fs,500)):
        print(res)